## PRÁCTICA MULTICASTING

Alejandro Manuel López Gómez

### Práctica 1.1 Multicast
Un participante se unirá al grupo, a continuación enviará un mensaje. Después se quedará a la espera de recibir mensajes de otros participantes, hasta que el mensaje recibido sea “Adiós”. (en ese momento todos los
participantes saldrán del grupo de multidifusión en bloque)

El siguiente código se emplea para enviar mensajes con la finalidad de probar el funcionamiento del "Adios" de la práctica 1.1. Si se ejecuta en User1, el resultado se deberá obervar en User2 y viceversa.

In [ ]:
import socket
import struct
import pickle

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR,1)

sock.bind(('',MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

ttl = struct.pack('b',1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

username = str(input("Input your username> "))

while True:
    msg = str(input(username + "> "))
    sock.sendto(pickle.dumps((username,msg)), (MCAST_GRP, MCAST_PORT))

Input your username> blanca
blanca> hola
blanca> que tal
blanca> pues yo me voy
blanca> Adios


In [3]:
import socket
import struct
import pickle

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR,1)

sock.bind(('',MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

ttl = struct.pack('b',1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

username = str(input("Input your username> "))
msg = str(input(username + "> "))

sock.sendto(pickle.dumps((username,msg)), (MCAST_GRP, MCAST_PORT))

print("Waiting for other participants...")

while True:
    res = pickle.loads(sock.recv(1024))
    
    # Evitar mostrar mensajes duplicados
    if (res[0] != username):
        print(res[0] + "> " + res[1])
    
    # En el momento en el que se reciba un "Adios" al ser todos receptores del mensaje todos los clientes
    # ejecutarán este condicional, finalizando la sesión grupal
    if (res[1] == "Adios"):
        print("Closing chat room...")
        break

sock.close()

Input your username> blanca
blanca> hola
Waiting for other participants...
alex> que tal
alex> todo bien?
alex> ok pues me voy
alex> Adios
Closing chat room...


### PRÁCTICA 1.2 Multicast
Se realizará una aplicación de chat utilizando la multidifusión IP. En este caso cada miembro
del grupo, podrá enviar sucesivamente tantos mensajes como desee al resto de participantes,
hasta que este decida salir del grupo con el mensaje “Quit”. (El resto de participantes seguirán
permaneciendo en el grupo de difusión, y solo saldrán de este cuando ellos mismo decidan
dejarlo mediante el mensaje “Quit”).

In [1]:
import threading
import socket
import struct
import pickle

stop = False

# Función de lectura de mensajes
def reader_func(sock,username):
    global stop
    try:
        while stop == False:
            res = pickle.loads(sock.recv(1024))

            if (res[0] != username):
                print(res[0] + "> " + res[1])
    except OSError:
        print("Stop reader")
        
MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR,1)

sock.bind(('',MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

ttl = struct.pack('b',1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

# Se pide al usuario su nombre y se crean los mensajes de bienvenida y despedida
username = str(input("Input your username> "))
welcome_msg = "User " + username + " has joined :D"
bye_msg = "User " + username + " has left :<"

# Se anuncia al grupo la llegada del usuario
sock.sendto(pickle.dumps((username,welcome_msg)), (MCAST_GRP, MCAST_PORT))

print(welcome_msg)

# Comienza el lector en una hebra
reader = threading.Thread(target=reader_func, args=(sock,username))
reader.start()

# Bucle principal de lectura
while stop == False:
    msg = str(input(username + "> "))

    # En el momento en el que el propio usuario escriba "Quit", este y solo este abandonará la comunicación
    if (msg == "Quit"):
        print("Leaving chat room")
        sock.sendto(pickle.dumps((username,bye_msg)), (MCAST_GRP, MCAST_PORT))
        
        # Al poner a True la variable stop finalizan tanto la hebra como el bucle principal
        stop = True
    else:
        sock.sendto(pickle.dumps((username,msg)), (MCAST_GRP, MCAST_PORT))

sock.close()

Input your username> blanca
User blanca has joined :D
blanca> holaaa
alex> como va todo?
blanca> muy bieeen y tu?
alex> todo bien
alex> bueno me voy
alex> User alex has left :<
blanca> ok pues yo tambien me voy
blanca> Quit
Leaving chat room
